In [ ]:
import polars as pl
from pathlib import Path


# load pareto sets
# problem_name = "branin"
problem_name = "mystery"
file_ext = "parquet"
n_gen_front = 1_000
psize_front = 100
# optimal_value = -268.78792  # branin
optimal_value = -1.174261  # mystery

fronts_path = Path("results/fronts/")
front_file_name = Path(fronts_path / f"{problem_name}_gen{n_gen_front}_psize{psize_front}.{file_ext}")

df_front = pl.read_parquet(front_file_name)
df_front.filter(pl.col("c_1") <= 0).sort("f_1")

x_1,x_2,f_1,c_1,f_1_min,c_1_min,generation
f64,f64,f64,f64,f64,f64,i32
3.280376,0.045728,-268.783606,-0.000413,-268.783606,-0.000413,502
3.299727,0.038661,-268.73532,-0.003805,-268.73532,-0.003805,785
3.301798,0.039454,-268.683851,-0.010478,-268.683851,-0.010478,487
3.250246,0.063265,-268.665227,-0.014868,-268.665227,-0.014868,478
3.23948,0.069027,-268.638584,-0.015528,-268.638584,-0.015528,504
…,…,…,…,…,…,…
3.146499,2.200212,-210.805058,-4.596961,-210.805058,-4.596961,663
3.149658,2.201208,-210.736264,-4.597243,-210.736264,-4.597243,503
3.133901,2.220426,-210.46082,-4.598159,-210.46082,-4.598159,950


In [9]:
f_col = "f_1_min"
c_col = "c_1_min"

front = df_front.select([f_col, c_col]).drop_nulls().filter(pl.col(f_col).is_finite() & pl.col(c_col).is_finite())

f_ideal, f_nadir = front.select(
    pl.min(f_col).alias("f_ideal"),
    pl.max(f_col).alias("f_nadir"),
)

front_norm = front.with_columns(
    ((pl.col(f_col) - f_ideal) / (f_nadir - f_ideal)).alias("f_norm"),
)

norm_optimal_value = (optimal_value - f_ideal) / (f_nadir - f_ideal)
norm_optimal_value

f_ideal
f64
0.754111


In [10]:
low_th_norm = norm_optimal_value - 0.01 * norm_optimal_value
med_th_norm = norm_optimal_value - 0.05 * norm_optimal_value
hig_th_norm = norm_optimal_value - 0.10 * norm_optimal_value

print(med_th_norm)

deltas = front_norm.with_columns(
    (abs(pl.col("f_norm") - low_th_norm)).alias("low_delta"),
    (abs(pl.col("f_norm") - med_th_norm)).alias("med_delta"),
    (abs(pl.col("f_norm") - hig_th_norm)).alias("hig_delta"),
)

low_ct = deltas.sort("low_delta").select(pl.first(c_col))
med_ct = deltas.sort("med_delta").select(pl.first(c_col))
hig_ct = deltas.sort("hig_delta").select(pl.first(c_col))

print(f"{low_ct=}\n, {med_ct=}\n, {hig_ct=}\n")


shape: (1,)
Series: 'f_ideal' [f64]
[
	0.716406
]
low_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 0.265414 │
└──────────┘
, med_ct=shape: (1, 1)
┌──────────┐
│ c_1_min  │
│ ---      │
│ f64      │
╞══════════╡
│ 3.051701 │
└──────────┘
, hig_ct=shape: (1, 1)
┌───────────┐
│ c_1_min   │
│ ---       │
│ f64       │
╞═══════════╡
│ 10.099325 │
└───────────┘

